# Emoji Prediction

In [1]:
!pip install emoji

  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49720 sha256=0ad201bbeca25008c55b5a9c28c648c75987d6ba4e351afa09c8b5b040b53414
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\43\3d\82\e7baffa5e86346c6178d7750dba6e8ef063282a37fc563f8f8
Successfully built emoji


You should consider upgrading via the 'c:\users\hp\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd


In [4]:
train = pd.read_csv('train_emoji.csv',header=None)
test = pd.read_csv('test_emoji.csv',header=None)

In [5]:
train.head()

0  1   2     3
0           never talk to me again  3 NaN   NaN
1  I am proud of your achievements  2 NaN   NaN
2   It is the worst day in my life  3 NaN   NaN
3                 Miss you so much  0 NaN   [0]
4                     food is life  4 NaN   NaN

In [6]:
train.tail()

0  1   2    3
127  he had to make a home run  1 NaN  NaN
128         I am ordering food  4 NaN  NaN
129     What is wrong with you  3 NaN  NaN
130                 I love you  0 NaN  NaN
131                  great job  2 NaN  NaN

In [7]:
import emoji as emoji

In [9]:
#emoji.EMOJI_ALIAS_UNICODE

In [10]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [12]:
emoji.emojize(":fork_and_knife:")

'🍴'

In [13]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [14]:
data = train.values
for i in range(10):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [19]:
print(data[100][0],emoji.emojize(emoji_dictionary[str(data[100][1])]))

I love my dad ❤️


In [20]:
from keras.utils import to_categorical

In [21]:
XT = train[0]
Xt = test[0]

YT = to_categorical(train[1])
Yt = to_categorical(test[1])


print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [26]:
embeddings = {}
with open('glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
       # print(word)
       # print(coeffs)
        embeddings[word] = coeffs

In [27]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [28]:
emb_XT = getOutputEmbeddings(XT)
emb_Xt = getOutputEmbeddings(Xt)

<ipython-input-27-73145de5bfac>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix] = X[ix].split()


In [29]:
print(emb_XT.shape)
print(emb_Xt.shape)


(132, 10, 50)
(56, 10, 50)


In [30]:
from keras.layers import *
from keras.models import Sequential

In [33]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [34]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(emb_XT,YT,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

Epoch 1/40
4/4 [==============================] - 1s 210ms/step - loss: 1.5966 - acc: 0.2288 - val_loss: 1.6126 - val_acc: 0.0714
Epoch 2/40
4/4 [==============================] - 0s 10ms/step - loss: 1.5348 - acc: 0.3136 - val_loss: 1.6160 - val_acc: 0.0714
Epoch 3/40
4/4 [==============================] - 0s 9ms/step - loss: 1.4762 - acc: 0.3814 - val_loss: 1.6078 - val_acc: 0.2857
Epoch 4/40
4/4 [==============================] - 0s 9ms/step - loss: 1.4233 - acc: 0.4407 - val_loss: 1.5865 - val_acc: 0.2857
Epoch 5/40
4/4 [==============================] - 0s 9ms/step - loss: 1.3650 - acc: 0.4237 - val_loss: 1.5195 - val_acc: 0.2857
Epoch 6/40
4/4 [==============================] - 0s 11ms/step - loss: 1.2979 - acc: 0.5424 - val_loss: 1.4129 - val_acc: 0.5000
Epoch 7/40
4/4 [==============================] - 0s 11ms/step - loss: 1.1759 - acc: 0.6186 - val_loss: 1.3756 - val_acc: 0.5000
Epoch 8/40
4/4 [==============================] - 0s 11ms/step - loss: 1.0876 - acc: 0.6356 - val_l

In [35]:
model.evaluate(emb_Xt,Yt)

2/2 [==============================] - 0s 35ms/step - loss: 1.4073 - acc: 0.5893


[1.407334566116333, 0.5892857313156128]

In [37]:
pred = model.predict_classes(emb_Xt)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [39]:
for i in range(50):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
😓
she got me a present
❤️
😁
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
😁
I am upset
❤️
😓
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
🍴
Stop making this joke ha ha ha
😁
😁
where is the ball
⚾
⚾
work is hard
😓
😁
This girl is messing with me
😓
❤️
are you serious ha ha
😁
❤️
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😁
Congratulation for having a baby
😁
😁
stop messing around
😓
⚾
any suggestions for dinner
🍴
🍴
I love taking breaks
❤️
❤️
you brighten my day
😁
❤️
I boiled rice
🍴
🍴
she is a bully
😓
😓
Why are you feeling bad
😓
😓
I am upset
😓
😓
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
⚾
valentine day is near
❤️
😁
I miss you so much
❤️
❤️
throw the ball
⚾
⚾
My life is so boring
😓
❤️
she said yes
😁
😁
will you be my valentine
❤️
😁
he can pitch really well
⚾
⚾
dance with me
😁
😁
I am starving
🍴
🍴
See you at the restaurant
🍴
🍴
